In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

In [2]:
# Part 1 - Data Preprocessing

In [7]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [10]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/home/ezequiel/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/ezequiel/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:450: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [11]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
# Part 2 - Now let's make the ANN!

In [14]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [15]:
# Initialising the ANN
classifier = Sequential()

In [17]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
# classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
8000/8000 [==============================] - 1s 138us/step - loss: 0.5494 - acc: 0.7955
Epoch 2/10
8000/8000 [==============================] - 1s 94us/step - loss: 0.5062 - acc: 0.7960
Epoch 3/10
8000/8000 [==============================] - 1s 96us/step - loss: 0.5059 - acc: 0.7960
Epoch 4/10
8000/8000 [==============================] - 1s 96us/step - loss: 0.5062 - acc: 0.7960
Epoch 5/10
8000/8000 [==============================] - 1s 93us/step - loss: 0.5062 - acc: 0.7960
Epoch 6/10
8000/8000 [==============================] - 1s 93us/step - loss: 0.5061 - acc: 0.7960
Epoch 7/10
8000/8000 [==============================] - 1s 96us/step - loss: 0.5062 - acc: 0.7960
Epoch 8/10
8000/8000 [==============================] - 1s 94us/step - loss: 0.5061 - acc: 0.7960
Epoch 9/10
8000/8000 [==============================] - 1s 101us/step - loss: 0.5061 - acc: 0.7960
Epoch 10/10
8000/8000 [==============================] - 1s 107us/step - loss: 0.5062 - acc: 0.7960


In [18]:
# Part 3 - Making predictions and evaluating the model

In [19]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [20]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)

In [21]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [22]:
print(cm)

[[1595    0]
 [ 405    0]]


In [23]:
# Part 4 - Evaluating, Improving and Tuning the ANN

In [24]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [25]:
print(accuracies)
print(mean)
print(variance)

[0.83375    0.82874999 0.8        0.82125    0.84749999 0.83875
 0.7875     0.81749999 0.81875    0.83249999]
0.8226249955967069
0.017106740588306368


In [26]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [27]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

In [28]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [29]:
classifier = KerasClassifier(build_fn = build_classifier)

In [30]:
parameters = {'batch_size': [10, 25, 50],
              'epochs': [10, 25, 50],
              'optimizer': ['adam', 'rmsprop']}

In [31]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [32]:
grid_search = grid_search.fit(X_train, y_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/10
7200/7200 [==============================] - 1s 145us/step - loss: 0.4908 - acc: 0.7972
Epoch 2/10
7200/7200 [==============================] - 1s 93us/step - loss: 0.4356 - acc: 0.7971
Epoch 3/10
7200/7200 [==============================] - 1s 93us/step - loss: 0.4299 - acc: 0.7971
Epoch 4/10
7200/7200 [==============================] - 1s 92us/step - loss: 0.4272 - acc: 0.7971
Epoch 5/10
7200/7200 [==============================] - 1s 96us/step - loss: 0.4237 - acc: 0.7971
Epoch 6/10
7200/7200 [==============================] - 1s 91us/step - loss: 0.4296 - acc: 0.8042
Epoch 7/10
7200/7200 [==============================] - 1s 93us/step - loss: 0.4269 - acc: 0.8179
Epoch 8/10
7200/7200 [==============================] - 1s 92us/step - loss: 0.4220 - acc: 0.8242
Epoch 9/10
7200/7200 [==============================] - 1s 93us/step - loss: 0.4255 - acc: 0.8246
Ep

7200/7200 [==============================] - 1s 103us/step - loss: 0.4245 - acc: 0.8289
Epoch 1/10
7200/7200 [==============================] - 1s 188us/step - loss: 0.4915 - acc: 0.7951
Epoch 2/10
7200/7200 [==============================] - 1s 102us/step - loss: 0.4319 - acc: 0.7957
Epoch 3/10
7200/7200 [==============================] - 1s 102us/step - loss: 0.4296 - acc: 0.7957
Epoch 4/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.4246 - acc: 0.7957
Epoch 5/10
7200/7200 [==============================] - 1s 102us/step - loss: 0.4253 - acc: 0.8003
Epoch 6/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.4223 - acc: 0.8219
Epoch 7/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.4236 - acc: 0.8257
Epoch 8/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.4249 - acc: 0.8231
Epoch 9/10
7200/7200 [==============================] - 1s 103us/step - loss: 0.4232 - acc: 0.8249
Epoch 10/10
7200/7200

7200/7200 [==============================] - 1s 116us/step - loss: 0.4391 - acc: 0.7972
Epoch 3/10
7200/7200 [==============================] - 1s 111us/step - loss: 0.4279 - acc: 0.8182
Epoch 4/10
7200/7200 [==============================] - 1s 110us/step - loss: 0.4192 - acc: 0.8181
Epoch 5/10
7200/7200 [==============================] - 1s 111us/step - loss: 0.4161 - acc: 0.8243
Epoch 6/10
7200/7200 [==============================] - 1s 111us/step - loss: 0.4082 - acc: 0.8226
Epoch 7/10
7200/7200 [==============================] - 1s 112us/step - loss: 0.4063 - acc: 0.8236
Epoch 8/10
7200/7200 [==============================] - 1s 113us/step - loss: 0.4060 - acc: 0.8264
Epoch 9/10
7200/7200 [==============================] - 1s 112us/step - loss: 0.4043 - acc: 0.8235
Epoch 10/10
7200/7200 [==============================] - 1s 110us/step - loss: 0.4057 - acc: 0.8281
Epoch 1/10
7200/7200 [==============================] - 2s 245us/step - loss: 0.5429 - acc: 0.7957
Epoch 2/10
7200/7200

7200/7200 [==============================] - 1s 127us/step - loss: 0.4271 - acc: 0.8301
Epoch 20/25
7200/7200 [==============================] - 1s 142us/step - loss: 0.4245 - acc: 0.8299
Epoch 21/25
7200/7200 [==============================] - 1s 153us/step - loss: 0.4227 - acc: 0.8306
Epoch 22/25
7200/7200 [==============================] - 1s 142us/step - loss: 0.4253 - acc: 0.8283
Epoch 23/25
7200/7200 [==============================] - 1s 152us/step - loss: 0.4252 - acc: 0.8318
Epoch 24/25
7200/7200 [==============================] - 1s 135us/step - loss: 0.4248 - acc: 0.8300
Epoch 25/25
7200/7200 [==============================] - 1s 138us/step - loss: 0.4252 - acc: 0.8325
Epoch 1/25
7200/7200 [==============================] - 2s 306us/step - loss: 0.5037 - acc: 0.7946
Epoch 2/25
7200/7200 [==============================] - 1s 137us/step - loss: 0.4361 - acc: 0.7956
Epoch 3/25
7200/7200 [==============================] - 1s 135us/step - loss: 0.4333 - acc: 0.7953
Epoch 4/25
7200

7200/7200 [==============================] - 2s 326us/step - loss: 0.5279 - acc: 0.7937
Epoch 2/25
7200/7200 [==============================] - 1s 137us/step - loss: 0.4593 - acc: 0.7944
Epoch 3/25
7200/7200 [==============================] - 1s 137us/step - loss: 0.4522 - acc: 0.7944
Epoch 4/25
7200/7200 [==============================] - 1s 138us/step - loss: 0.4505 - acc: 0.7944
Epoch 5/25
7200/7200 [==============================] - 1s 136us/step - loss: 0.4492 - acc: 0.7944
Epoch 6/25
7200/7200 [==============================] - 1s 138us/step - loss: 0.4455 - acc: 0.7944
Epoch 7/25
7200/7200 [==============================] - 1s 136us/step - loss: 0.4520 - acc: 0.7944
Epoch 8/25
7200/7200 [==============================] - 1s 139us/step - loss: 0.4488 - acc: 0.7944
Epoch 9/25
7200/7200 [==============================] - 1s 140us/step - loss: 0.4477 - acc: 0.7944
Epoch 10/25
7200/7200 [==============================] - 1s 142us/step - loss: 0.4443 - acc: 0.7944
Epoch 11/25
7200/720

...

7200/7200 [==============================] - 1s 101us/step - loss: 0.4245 - acc: 0.8199
Epoch 17/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4228 - acc: 0.8215
Epoch 18/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4257 - acc: 0.8235
Epoch 19/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4226 - acc: 0.8264
Epoch 20/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4242 - acc: 0.8254
Epoch 21/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4225 - acc: 0.8274
Epoch 22/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4245 - acc: 0.8285
Epoch 23/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4220 - acc: 0.8276
Epoch 24/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4251 - acc: 0.8286
Epoch 25/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4208 - acc: 0.8290
Epoch 26/50


7200/7200 [==============================] - 1s 101us/step - loss: 0.4266 - acc: 0.8251
Epoch 49/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4304 - acc: 0.8258
Epoch 50/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4307 - acc: 0.8268
Epoch 1/50
7200/7200 [==============================] - 13s 2ms/step - loss: 0.6534 - acc: 0.7917
Epoch 2/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.5302 - acc: 0.7944
Epoch 3/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4579 - acc: 0.7944
Epoch 4/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4468 - acc: 0.7944
Epoch 5/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4435 - acc: 0.7944
Epoch 6/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4383 - acc: 0.7944
Epoch 7/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4396 - acc: 0.7944
Epoch 8/50
7200/7200

7200/7200 [==============================] - 1s 104us/step - loss: 0.4276 - acc: 0.8287
Epoch 31/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4256 - acc: 0.8289
Epoch 32/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4263 - acc: 0.8319
Epoch 33/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4281 - acc: 0.8290
Epoch 34/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4252 - acc: 0.8278
Epoch 35/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4226 - acc: 0.8310
Epoch 36/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4237 - acc: 0.8293
Epoch 37/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4251 - acc: 0.8293
Epoch 38/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4260 - acc: 0.8293
Epoch 39/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4222 - acc: 0.8301
Epoch 40/50


7200/7200 [==============================] - 1s 103us/step - loss: 0.4317 - acc: 0.7957
Epoch 13/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4272 - acc: 0.7957
Epoch 14/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4295 - acc: 0.7957
Epoch 15/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4287 - acc: 0.7957
Epoch 16/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4258 - acc: 0.7957
Epoch 17/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4286 - acc: 0.7957
Epoch 18/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4267 - acc: 0.7957
Epoch 19/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4221 - acc: 0.7957
Epoch 20/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4249 - acc: 0.7957
Epoch 21/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4265 - acc: 0.7957
Epoch 22/50


7200/7200 [==============================] - 1s 102us/step - loss: 0.4143 - acc: 0.8292
Epoch 45/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.4145 - acc: 0.8303
Epoch 46/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4174 - acc: 0.8256
Epoch 47/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4165 - acc: 0.8293
Epoch 48/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4193 - acc: 0.8235
Epoch 49/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4172 - acc: 0.8282
Epoch 50/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4120 - acc: 0.8300
Epoch 1/50
8000/8000 [==============================] - 15s 2ms/step - loss: 0.5557 - acc: 0.7961
Epoch 2/50
8000/8000 [==============================] - 2s 206us/step - loss: 0.4421 - acc: 0.7960
Epoch 3/50
8000/8000 [==============================] - 2s 206us/step - loss: 0.4337 - acc: 0.7960
Epoch 4/50
8000/

In [38]:
best_parameters = grid_search.best_params_

In [39]:
best_accuracy = grid_search.best_score_

In [40]:
print(best_parameters)

{'batch_size': 25, 'epochs': 50, 'optimizer': 'rmsprop'}


In [42]:
print(best_accuracy)

0.8355
